# Input and output formats

This section provides an overview of the input and output formats
supported by DFTK, usually via integration with a third-party library.

## Reading / writing files supported by AtomsIO
[AtomsIO](https://github.com/mfherbst/AtomsIO.jl) is a Julia package which supports
reading / writing atomistic structures from / to a large range of file formats.
Supported formats include Crystallographic Information Framework (CIF),
XYZ and extxyz files, ASE / Gromacs / LAMMPS / Amber trajectory files
or input files of various other codes (e.g. Quantum Espresso, VASP, ABINIT, CASTEP, …).
The full list of formats is is available in the
[AtomsIO documentation](https://mfherbst.github.io/AtomsIO.jl/stable).

The AtomsIO functionality is split into two packages. The main package, `AtomsIO` itself,
only depends on packages, which are registered in the Julia General package registry.
In contrast `AtomsIOPython` extends `AtomsIO` by parsers depending on python packages,
which are automatically managed via `PythonCall`. While it thus provides the full set of
supported IO formats, this also adds additional practical complications, so some users
may choose not to use `AtomsIOPython`.

As an example we start the calculation of a simple antiferromagnetic iron crystal
using a Quantum-Espresso input file, [Fe_afm.pwi](Fe_afm.pwi).
For more details about calculations on magnetic systems
using collinear spin, see Collinear spin and magnetic systems.

First we parse the Quantum Espresso input file using AtomsIO,
which reads the lattice, atomic positions and initial magnetisation
from the input file and returns it as an
[AtomsBase](https://github.com/JuliaMolSim/AtomsBase.jl) `AbstractSystem`,
the JuliaMolSim community standard for representing atomic systems.

In [1]:
using AtomsIO        # Use Julia-only IO parsers
using AtomsIOPython  # Use python-based IO parsers (e.g. ASE)
system = load_system("Fe_afm.pwi")

FlexibleSystem(Fe₂, periodic = TTT):
    bounding_box      : [ 2.86814        0        0;
                                0  2.86814        0;
                                0        0  2.86814]u"Å"

    Atom(Fe, [       0,        0,        0]u"Å")
    Atom(Fe, [ 1.43407,  1.43407,        0]u"Å")

      .------.  
     /|      |  
    * |      |  
    | |      |  
    | .------.  
    |/  Fe  /   
    Fe-----*    


Next we attach pseudopotential information, since currently the parser is not
yet capable to read this information from the file.

In [2]:
using DFTK
system = attach_psp(system, Fe="hgh/pbe/fe-q16.hgh")

FlexibleSystem(Fe₂, periodic = TTT):
    bounding_box      : [ 2.86814        0        0;
                                0  2.86814        0;
                                0        0  2.86814]u"Å"

    Atom(Fe, [       0,        0,        0]u"Å")
    Atom(Fe, [ 1.43407,  1.43407,        0]u"Å")

      .------.  
     /|      |  
    * |      |  
    | |      |  
    | .------.  
    |/  Fe  /   
    Fe-----*    


Finally we make use of DFTK's AtomsBase integration to run the calculation.

In [3]:
model = model_LDA(system; temperature=0.01)
basis = PlaneWaveBasis(model; Ecut=10, kgrid=(2, 2, 2))
ρ0 = guess_density(basis, system)
scfres = self_consistent_field(basis, ρ=ρ0);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ------
  1   -223.7493312456                    0.22   -6.320    5.5         
  2   -224.1605916662       -0.39       -0.21   -3.325    2.0    304ms
  3   -224.2175970871       -1.24       -1.06   -1.615    2.9    292ms
  4   -224.2198751612       -2.64       -1.46   -1.192    1.1    210ms
  5   -224.2207938951       -3.04       -1.72   -0.823    1.0    212ms
  6   -224.2212121247       -3.38       -2.00   -0.486    1.0    206ms
  7   -224.2213739798       -3.79       -2.35   -0.228    1.9    266ms
  8   -224.2214155717       -4.38       -2.88   -0.076    1.9    219ms
  9   -224.2214205624       -5.30       -3.56   -0.007    2.8    251ms
 10   -224.2214206904       -6.89       -3.81    0.001    2.9    276ms
 11   -224.2214207104       -7.70       -4.05   -0.001    2.7    272ms
 12   -224.2214207181       -8.11       -4.48    0.000    1.7    225ms
 13   -

## Writing VTK files for visualization
For visualizing the density or the Kohn-Sham orbitals DFTK supports storing
the result of an SCF calculations in the form of VTK files.
These can afterwards be visualized using tools such
as [paraview](https://www.paraview.org/).
Using this feature requires
the [WriteVTK.jl](https://github.com/jipolanco/WriteVTK.jl/) Julia package.

In [4]:
using WriteVTK
save_scfres("iron_afm.vts", scfres; save_ψ=true);

This will save the iron calculation above into the file `iron_afm.vts`,
using `save_ψ=true` to also include the KS orbitals.

## Writing and reading JLD2 files
The full state of a DFTK self-consistent field calculation can be
stored on disk in form of an [JLD2.jl](https://github.com/JuliaIO/JLD2.jl) file.
This file can be read from other Julia scripts
as well as other external codes supporting the HDF5 file format
(since the JLD2 format is based on HDF5).

In [5]:
using JLD2
save_scfres("iron_afm.jld2", scfres);

Since such JLD2 can also be read by DFTK to start or continue a calculation,
these can also be used for checkpointing or for transferring results
to a different computer.
See Saving SCF results on disk and SCF checkpoints for details.

(Cleanup files generated by this notebook.)

In [6]:
rm("iron_afm.vts")
rm("iron_afm.jld2")